# Laden der VBN GTFS-R Daten

In [ ]:
import pandas as pd
import os
import duckdb
from google.transit import gtfs_realtime_pb2
import requests


In [ ]:
url = "http://gtfsr.vbn.de/gtfsr_connect.bin"

In [ ]:

feed = gtfs_realtime_pb2.FeedMessage()
response = requests.get(url)
feed.ParseFromString(response.content)
for entity in feed.entity:
  if entity.HasField('trip_update'):
    print(entity.trip_update)